In [3]:
import numpy as np
import pandas as pd
import folium as fo
from folium.plugins import HeatMap
import geopandas as gpd

NextBike = pd.read_csv("touren_Nextbike.csv")

In [4]:
NextBike["No_Move"]=(NextBike["start_lat"]==NextBike["end_lat"])&(NextBike["start_lon"]==NextBike["end_lon"]) 
Clean_NextBike = NextBike[NextBike["No_Move"]==False]

In [5]:
filepath = "all_routes_graphhopper_local.geojson"
Graphhopperroutes = gpd.read_file(filepath)
Graphhopperroutes_coordinates = pd.DataFrame()
Graphhopperroutes_coordinates["GraphhopperKoordinaten"] = None
Graphhopperroutes = Graphhopperroutes[Graphhopperroutes.geometry.notna()] # delete entries where the geometry object is NaN

for i in range(0,len(Graphhopperroutes)):
    Graphhopperroutes_coordinates.loc[i,"GraphhopperKoordinaten"] = Graphhopperroutes.iloc[i].geometry.coords[:]
Graphhopperroutes_coordinates


,GraphhopperKoordinaten
0,"[(8.482362, 49.47032), (8.482098, 49.470511), ..."
1,"[(8.46976, 49.479971), (8.469592, 49.480037), ..."
2,"[(8.46976, 49.479971), (8.469592, 49.480037), ..."
3,"[(8.481132, 49.456729), (8.480707, 49.457313),..."
4,"[(8.46332, 49.482771), (8.463189, 49.482823), ..."
...,...
4523,"[(8.503815, 49.467546), (8.503737, 49.467576),..."
4524,"[(8.476644, 49.485352), (8.475703, 49.485645),..."
4525,"[(8.537512, 49.451781), (8.537476, 49.451794),..."
4526,"[(8.486921, 49.474336), (8.48699, 49.474432), ..."


In [6]:
Graphhopperroutes_coordinates["startpoint_lat"] = None
Graphhopperroutes_coordinates["startpoint_lon"] = None
Graphhopperroutes_coordinates["endpoint_lat"] = None
Graphhopperroutes_coordinates["endpoint_lon"] = None

In [7]:
colIndex_startLat = Graphhopperroutes_coordinates.columns.get_loc("startpoint_lat")
colIndex_startLon = Graphhopperroutes_coordinates.columns.get_loc("startpoint_lon")
colIndex_endLat = Graphhopperroutes_coordinates.columns.get_loc("endpoint_lat")
colIndex_endLon = Graphhopperroutes_coordinates.columns.get_loc("endpoint_lon")

for k in range(0,len(Graphhopperroutes_coordinates)):
    startlat_k = Graphhopperroutes_coordinates.iloc[k].iloc[0][0][1]
    startlon_k = Graphhopperroutes_coordinates.iloc[k].iloc[0][0][0] 
    endlat_k = Graphhopperroutes_coordinates.iloc[k].iloc[0][-1][1]
    endlon_k = Graphhopperroutes_coordinates.iloc[k].iloc[0][-1][0]
    Graphhopperroutes_coordinates.iloc[k, colIndex_startLat] = startlat_k
    Graphhopperroutes_coordinates.iloc[k, colIndex_startLon] = startlon_k
    Graphhopperroutes_coordinates.iloc[k,colIndex_endLat] = endlat_k
    Graphhopperroutes_coordinates.iloc[k,colIndex_endLon] = endlon_k

In [11]:
Clean_NextBike

,Unnamed: 0,FahrradID,AusleihstationID,AusleihstationName,AusleihstationPLZ,AusleihzeitID,RueckgabestationID,RueckgabestationName,RueckgabestationPLZ,Rueckgabezeit,Dauer,Rueckgabe_datetime,Rueckgabe_Stunde,Rueckgabe_Stunde_des_Tages,start_lat,start_lon,end_lat,end_lon,dist_km,No_Move
1,1,42730,339475,Hauptbahnhof Mannheim,68001,120195,339343,Neckarstadt - Alter Messplatz,68169.0,120206,11,2025-05-28 23:10:00,2025-05-28 23:00:00,23,49.479960,8.469749,49.497070,8.473044,1.917374,False
2,2,42730,339343,Neckarstadt - Alter Messplatz,68169,120235,339466,G7 - Westliche Unterstadt,68159.0,120246,11,2025-05-28 23:50:00,2025-05-28 23:00:00,23,49.497070,8.473044,49.491183,8.462391,1.010192,False
3,3,42730,339466,G7 - Westliche Unterstadt,68159,120725,148606240,A1 - Landgericht/Universität,68002.0,120736,11,2025-05-29 08:00:00,2025-05-29 08:00:00,8,49.491183,8.462391,49.484796,8.462552,0.710297,False
4,4,42730,148606240,A1 - Landgericht/Universität,68002,120745,339472,A5 - Universität West,68002.0,120746,1,2025-05-29 08:10:00,2025-05-29 08:00:00,8,49.484796,8.462552,49.487110,8.456876,0.484062,False
5,5,42730,339472,A5 - Universität West,68002,120815,339354,K1 - Kurpfalzbrücke,68169.0,120846,31,2025-05-29 09:50:00,2025-05-29 09:00:00,9,49.487110,8.456876,49.493002,8.470325,1.171706,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381324,381324,44464,339462,R1 - Marktplatz,68161,131352,121376,Lindenhof - Jugendherberge,68003.0,131383,31,2025-06-05 17:27:00,2025-06-05 17:00:00,17,49.489761,8.468437,49.477082,8.463662,1.451426,False
381325,381325,44464,121376,Lindenhof - Jugendherberge,68003,131512,276163348,Tunnelstraße / Suezkanal,68001.0,131543,31,2025-06-05 20:07:00,2025-06-05 20:00:00,20,49.477082,8.463662,49.480781,8.469247,0.576182,False
381326,381326,44464,276163348,Tunnelstraße / Suezkanal,68001,131692,30612250,U3 – Herschelbad,68161.0,131703,11,2025-06-05 22:47:00,2025-06-05 22:00:00,22,49.480781,8.469247,49.490800,8.471387,1.124736,False
381327,381327,44464,30612250,U3 – Herschelbad,68161,131812,339350,Studentensiedlung Ludwig Frank,68167.0,131833,21,2025-06-06 00:57:00,2025-06-06 00:00:00,0,49.490800,8.471387,49.506903,8.492206,2.338080,False


In [44]:
Stations = Clean_NextBike[["AusleihstationID","AusleihstationName","start_lat","start_lon"]]
Stations.rename(columns={"AusleihstationID":"StationsID","AusleihstationName":"StationsName","start_lat":"lat","start_lon":"lon"},inplace=True)
Stations

,StationsID,StationsName,lat,lon
1,339475,Hauptbahnhof Mannheim,49.479960,8.469749
2,339343,Neckarstadt - Alter Messplatz,49.497070,8.473044
3,339466,G7 - Westliche Unterstadt,49.491183,8.462391
4,148606240,A1 - Landgericht/Universität,49.484796,8.462552
5,339472,A5 - Universität West,49.487110,8.456876
...,...,...,...,...
381324,339462,R1 - Marktplatz,49.489761,8.468437
381325,121376,Lindenhof - Jugendherberge,49.477082,8.463662
381326,276163348,Tunnelstraße / Suezkanal,49.480781,8.469247
381327,30612250,U3 – Herschelbad,49.490800,8.471387


In [59]:
Graphhopperroutes_coordinates["StartStationID"] = None
StartIDindex = Graphhopperroutes_coordinates.columns.get_loc("StartStationID")
Graphhopperroutes_coordinates["StartStationName"] = None
StartNameindex = Graphhopperroutes_coordinates.columns.get_loc("StartStationName")
Graphhopperroutes_coordinates["EndStationID"] = None
EndIDindex = Graphhopperroutes_coordinates.columns.get_loc("EndStationID")
Graphhopperroutes_coordinates["EndStationName"] = None
EndNameindex = Graphhopperroutes_coordinates.columns.get_loc("EndStationName")

for l in range(0,len(Graphhopperroutes_coordinates)):
    Stations_diffcalc = pd.DataFrame(np.copy(Stations.values), columns=Stations.columns)
    Stations_diffcalc["lat"] = abs(Stations_diffcalc["lat"] - Graphhopperroutes_coordinates.iloc[l,colIndex_startLat])
    Stations_diffcalc["lon"] = abs(Stations_diffcalc["lon"] - Graphhopperroutes_coordinates.iloc[l,colIndex_startLon])

    indexMinLat_start = Stations_diffcalc["lat"].idxmin()
    indexMinLon_start = Stations_diffcalc["lon"].idxmin()

    if indexMinLat_start==indexMinLon_start:
        Graphhopperroutes_coordinates.iloc[l,StartIDindex] = Stations["StationsID"].iloc[indexMinLat_start]
        Graphhopperroutes_coordinates.iloc[l,StartNameindex] = Stations["StationsName"].iloc[indexMinLat_start]

    Stations_diffcalc = pd.DataFrame(np.copy(Stations.values), columns=Stations.columns)
    Stations_diffcalc["lat"] = abs(Stations_diffcalc["lat"] - Graphhopperroutes_coordinates.iloc[l,colIndex_endLat])
    Stations_diffcalc["lon"] = abs(Stations_diffcalc["lon"] - Graphhopperroutes_coordinates.iloc[l,colIndex_endLon])

    indexMinLat_end = Stations_diffcalc["lat"].idxmin()
    indexMinLon_end = Stations_diffcalc["lon"].idxmin()

    if indexMinLat_end==indexMinLon_end:
        Graphhopperroutes_coordinates.iloc[l,EndIDindex] = Stations["StationsID"].iloc[indexMinLat_end]
        Graphhopperroutes_coordinates.iloc[l,EndNameindex] = Stations["StationsName"].iloc[indexMinLat_end]

Graphhopperroutes_coordinates

,GraphhopperKoordinaten,startpoint_lat,startpoint_lon,endpoint_lat,endpoint_lon,StartStationID,StartStationName,EndStationID,EndStationName
0,"[(8.482362, 49.47032), (8.482098, 49.470511), ...",49.47032,8.482362,49.477824,8.467846,339478,Hochschule Mannheim,364019650,Lindenhofplatz / HBF Süd
1,"[(8.46976, 49.479971), (8.469592, 49.480037), ...",49.479971,8.46976,49.487121,8.45685,339475,Hauptbahnhof Mannheim,339472,A5 - Universität West
2,"[(8.46976, 49.479971), (8.469592, 49.480037), ...",49.479971,8.46976,49.482771,8.46332,339475,Hauptbahnhof Mannheim,660951,Universität Schloss
3,"[(8.481132, 49.456729), (8.480707, 49.457313),...",49.456729,8.481132,49.477824,8.467846,660971,Neckarau - Hans-Sachs-Ring,364019650,Lindenhofplatz / HBF Süd
4,"[(8.46332, 49.482771), (8.463189, 49.482823), ...",49.482771,8.46332,49.487121,8.45685,660951,Universität Schloss,339472,A5 - Universität West
...,...,...,...,...,...,...,...,...,...
4523,"[(8.503815, 49.467546), (8.503737, 49.467576),...",49.467546,8.503815,49.486105,8.530659,24559100,Neuhermsheim - Dragonerweg,378595589,Feudenheim - St. Peter & Paul Kirche
4524,"[(8.476644, 49.485352), (8.475703, 49.485645),...",49.485352,8.476644,49.524994,8.481515,339461,Kongresszentrum Rosengarten,57585097,S-Bahnhof Waldhof
4525,"[(8.537512, 49.451781), (8.537476, 49.451794),...",49.451781,8.537512,49.467546,8.503815,378596231,Seckenheim - S-Bahnhof Seckenheim-Hochstätt,24559100,Neuhermsheim - Dragonerweg
4526,"[(8.486921, 49.474336), (8.48699, 49.474432), ...",49.474336,8.486921,49.521549,8.525375,660943,Schwetzingerstadt - rnv Betriebshof Möhlstraße,393329858,Franklin - Wasserwerkstraße


,GraphhopperKoordinaten,startpoint_lat,startpoint_lon,endpoint_lat,endpoint_lon,StartStationID,StartStationName,EndStationID,EndStationName
